In [1]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, widgets

In [2]:
 def update_layout_of_graph(fig: go.Figure,title: str = 'Plot')->go.Figure:
    fig.update_layout(
        width=800,
        height=600,
        autosize=False,
        plot_bgcolor='rgba(0,0,0,0)',
        title=title,
        
    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                      xaxis_title = 'input values',
                      yaxis_title = 'output values',
                      legend=dict(yanchor="top",
                                  y=0.9,
                                  xanchor="right",
                                  x=0.95),
                      title={
                          'x': 0.5,
                          'xanchor': 'center'
                      })
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
    return fig
def line_scatter(
    visible: bool = True,
    x_lines: np.array = np.array([]),
    y_lines: np.array = np.array([]),
    name_line: str = 'Predicted function',
    showlegend: bool = True,
) -> go.Scatter:
    # Adding the lines
    return go.Scatter(
        visible=visible,
        line=dict(color="blue", width=2),
        x=x_lines,
        y=y_lines,
        name=name_line,
        showlegend= showlegend
    )
def dot_scatter(
    visible: bool = True,
    x_dots: np.array = np.array([]),
    y_dots: np.array = np.array([]),
    name_dots: str = 'Observed points',
    showlegend: bool = True
) -> go.Scatter:
    # Adding the dots
    return go.Scatter(
        x=x_dots,
        visible=visible,
        y=y_dots,
        mode="markers",
        name=name_dots,
        marker=dict(color='red', size=8),
        showlegend=showlegend
    )
def uncertainty_area_scatter(
        visible: bool = True,
        x_lines: np.array = np.array([]),
        y_upper: np.array = np.array([]),
        y_lower: np.array = np.array([]),
        name: str = "mean plus/minus standard deviation",
) -> go.Scatter:

    return go.Scatter(
        visible=visible,
        x=np.concatenate((x_lines, x_lines[::-1])),  # x, then x reversed
        # upper, then lower reversed
        y=np.concatenate((y_upper, y_lower[::-1])),
        fill='toself',
        fillcolor='rgba(189,195,199,0.5)',
        line=dict(color='rgba(200,200,200,0)'),
        hoverinfo="skip",
        showlegend=True,
        name= name,
    )

In [3]:
class SquaredExponentialKernel:
    def __init__(self,length=1,sigma_f=1):
        self.length = length
        self.sigma_f = sigma_f
        
    def __call__(self, argument_1,argument_2):
        return float(self.sigma_f*np.exp(-np.linalg.norm(argument_1-argument_2)**2/(2*self.length**2)))

In [4]:
kernel = SquaredExponentialKernel(length=1)
x_lines = np.arange(-10, 10, 0.1)
fig0 = go.FigureWidget(data=[
    line_scatter(
        x_lines=x_lines,
        y_lines=np.array([kernel(x,0) for x in x_lines]),
    )
])

fig0 = update_layout_of_graph(fig0, title='Squared exponential kernel')


@interact(length=(0.1, 3, 0.1), argument_2=(-10, 10, 0.1))
def update(length=1, argument_2=0):
    with fig0.batch_update():
        kernel = SquaredExponentialKernel(length=length)
        fig0.data[0].y = np.array([kernel(x, argument_2) for x in x_lines])
        
fig0

interactive(children=(FloatSlider(value=1.0, description='length', max=3.0, min=0.1), FloatSlider(value=0.0, d…

FigureWidget({
    'data': [{'line': {'color': 'blue', 'width': 2},
              'name': 'Predicted function'…

In [5]:
def calculate_covariance_matrix(x1,x2,covariance_function):
    return np.array([[covariance_function(a,b) for a in x1] for b in x2])

In [6]:
class GPR:
    def __init__(self,
                 data_x,
                 data_y,
                 covariance_function=SquaredExponentialKernel(),
                 noise=0.0):
        self.data_x = data_x
        self.data_y = data_y
        self.covariance_function = covariance_function
        self.noise = noise
        self._memory = None
        
        self.inv_cov_matrix_of_input_data = np.linalg.inv(
            calculate_covariance_matrix(data_x, data_x, covariance_function) \
            + (noise+1e-7)*np.identity(len(data_x)))

    def predict(self,at_values):
        k_lower_star = calculate_covariance_matrix(self.data_x,at_values,self.covariance_function)
        k_lower_star2 = calculate_covariance_matrix(at_values,at_values,self.covariance_function)
        
        mean_at_values = np.dot(
            k_lower_star, 
            np.dot(self.data_y, 
                   self.inv_cov_matrix_of_input_data.T).T).flatten()
        
        covar_matrix = k_lower_star2 - \
            np.dot(k_lower_star, np.dot(
                   self.inv_cov_matrix_of_input_data, k_lower_star.T))
        
        variance = np.diag(covar_matrix)
        self._memory = {'mean': mean_at_values, 'covariance_matrix': covar_matrix, 'variance':variance}
                        
        return mean_at_values

In [7]:
# Function to show the summary of the fit
def summary(gp):
    optimized = gp.optimizer != None
    if not optimized:
        s1 = "Fitted Kernel(not optimized)\n\n%s" % gp.kernel_
    else:
        s1 = "Fitted Kernel(Optimized)\n\n%s" % gp.kernel_
    s2 = "\n\nlog marginal likelihood: %.5f" % gp.log_marginal_likelihood(gp.kernel_.theta)
    print(s1 + s2 + '\n')

In [8]:
from save_dataset import load_dict
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel, RBF

data = load_dict('data/trajectory.pkl')

#print(data)
v = data['v'][1:]
v_pred = data['v_pred'][1:]



a_error = (v - v_pred)/data['dt']


x_values = np.concatenate((data['p'][1:,:], data['q'][1:,:], data['v'][1:,:], data['w'][1:,:]),axis=1)
y_values = a_error[:,0] # just x dimension

n_sub = 1000
x_subsampled = x_values[::n_sub]
y_subsampled = y_values[::n_sub]
n_sampled = x_subsampled.shape[0]

print(n_sampled)
print(x_subsampled.shape)

print(y_subsampled.shape)
print(np.eye(n_sampled))

10
(10, 13)
(10,)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [9]:


kernel = ConstantKernel() + Matern(length_scale=2, nu=3/2) + WhiteKernel(noise_level=9)
kernel = RBF(1)

gp = gaussian_process.GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0)
gp.fit(x_subsampled, y_subsampled)

summary(gp)

x_pred = np.ones((13)).reshape(1,-1)
  
y_pred, sigma = gp.predict(x_values, return_std=True)
#print(y_pred)
#print(sigma)
rms = np.sqrt(np.sum(np.square(y_values - y_pred))/y_values.shape[0])

print(rms)

params = gp.get_params(True)
print(gp.score(x_subsampled, y_subsampled))
#print(params)
#print(params['copy_X_train'])
#print(gp.kernel.hyperparameters)
#print(gp.kernel_.get_params())

Fitted Kernel(Optimized)

RBF(length_scale=12.9)

log marginal likelihood: -4.88083

1.8476016698696858
1.0


In [10]:
gp.kernel_.k1

AttributeError: 'RBF' object has no attribute 'k1'

In [ ]:

print(kernel)

In [ ]:
y_pred, sigma = gp.predict(x_pred, return_std=True)

In [ ]:
x = np.arange(0,20,0.01)


In [ ]:

def plot_GPR(data_x, data_y, model, x, visible=True) -> list:
    
    mean = model.predict(x)
    std = np.sqrt(model._memory['variance'])
    data = []
    print(std)
    for i in range(1,4):
        data.append(
            uncertainty_area_scatter(
                x_lines=x,
                y_lower=mean-i*std,
                y_upper=mean+i*std,
                name=f"mean +- {i}*std",
                visible=visible))
    data.append(line_scatter(x_lines=x, y_lines=mean, visible=visible))
    data.append(dot_scatter(x_dots=data_x, y_dots=data_y, visible=visible))
    return data

In [ ]:
model = GPR(x_subsampled, y_subsampled, noise=0.1)


data = plot_GPR(data_x = x_subsampled, data_y=y_subsampled, x=x, model=model)
fig = go.Figure(data=data)
fig = update_layout_of_graph(fig=fig,
                             title='GPR with length 1, sigma0 and noise 0')
fig